In [ ]:
from Advertising.learners.greedyLearner import *
import matplotlib.pyplot as plt
import numpy as np
from SocialNetwork.social_network_environment import *
from SocialNetwork.cascade import *
from Advertising.enviroment.bid import *
from Advertising.enviroment.VCG import *
from Advertising.learners.GTS_Learner import *
from Advertising.learners.GTS_SW_Learner import *
from Advertising.learners.TS_no_qualities import *
from tqdm.autonotebook import tqdm
import itertools
seed = 6789
random.seed(seed)
only_first = False


lambdas = [0.5, 0.42, 0.38, 0.30, 0.2, 0.05]
social_network = social_network_environment()
social_network.init_from_json(
    json_path_network='Config/network.json', json_path_features='Config/features.json')

adversary_bids = []
for _ in range(0, 10):
    adversary_i_bids = []
    for __ in range(0, 5):
        adversary_i_bids.append(random.randint(0, 4))
    adversary_bids.append(adversary_i_bids)


with open("Config/estimated_influence.json", "r"):
    data = json.load(open("Config/estimated_influence.json"))

active_by_influence_reward = np.array(data.get("estimated_influence"))


In [ ]:
def test_sw(learner, arms, adversary_bids, only_first, active_by_influence_reward, n_rounds, lambdas, social_network, sw_size):
    clairvoyants = []
    clair_ex = []
    vcg = VCG(lambdas)
    number_of_pulls = [0]*len(arms)
    sum_expected_values = [0]*len(arms)
    expected_values = [0]*len(arms)
    thing_to_plot = []
    thing_to_plot.append(0)
    t = 0
    for i in range(0, n_rounds):
        t += 1
        arm_learner, pulled_arm = learner.pull_arm()

        if t > sw_size:
            adversary_bids = []
            for _ in range(0, 10):
                adversary_i_bids = []
                for __ in range(0, 5):
                    adversary_i_bids.append(random.randint(0, 4))
                adversary_bids.append(adversary_i_bids)
            clairvoyants.append(np.argmax(expected_values))
            clair_ex.append(expected_values[np.argmax(expected_values)])
            t = 0

        all_bids = []
        all_bids.append(arm_learner)
        for element in adversary_bids:
            all_bids.append(element)
        ad_allocation_list = setup(bids=all_bids, n_bids=5)

        best_allocation = vcg.all_best_allocations(
            list_camp_bids=ad_allocation_list, social_network=social_network)

        if only_first:
            for j in range(0, len(best_allocation)):
                temp_allocation = []
                temp_allocation = best_allocation[j]
                for k in range(0, len(best_allocation[j])):
                    if temp_allocation[k].ad_id == 1 and k != 0:
                        temp_allocation[k].ad_id = None
                        best_allocation[j] = temp_allocation

        total_reward, active_nodes = activate_cascade(
            social_network=social_network, ad_allocation_list=best_allocation, slot_prominence=deltas)

        payments = vcg.payments(
            ad_allocation_list, best_allocation, social_network=social_network)
    # with this part of code i set that if the learner is present in the allocation list then get the normal allocation
    # reward, if it's not present and it bids 0 then the reward is 0.

        payments_tot = calculate_total_payment(
            payments, social_network.categories, active_nodes)

        reward = total_reward - payments_tot

        if i > 0:
            thing_to_plot.append((reward+thing_to_plot[-1]*(i-1))/i)

        #rew = reward-np.sum(payments_tot)

        number_of_pulls[pulled_arm] += 1
        sum_expected_values[pulled_arm] += reward
        expected_values[pulled_arm] = sum_expected_values[pulled_arm] / \
            number_of_pulls[pulled_arm]

    # updating of the learners
        learner.update(pulled_arm, reward, number_of_pulls)
        reset_nodes(social_network=social_network)

    best_arm_index = np.argmax(expected_values)

    print(expected_values)
    print(best_arm_index)

    return arms[best_arm_index], best_arm_index, number_of_pulls, expected_values, thing_to_plot, clairvoyants, clair_ex


In [ ]:
expected_values = []
expected_values2 = []
sw_size = 3000
arms = arms_creation(seed=30291, number_of_arms=20)
ts_learner = GTS_SW_Learner(20, arms, variance=20, size_SW=sw_size, ad_id=1)
ts_learner2 = GTS_SW_Learner(20, arms, variance=20, size_SW=sw_size, ad_id=1)

for j in range(0, 1):
    best_Arm = []
    print("This is TS number " + str(j))

    best_arm, best_arm_index, number_of_pulls, expected_values, thing_to_plot, clairvoyants, clair_ex = test_sw(
        ts_learner, arms, adversary_bids, only_first, active_by_influence_reward, 10000, lambdas, social_network, sw_size)
    best_arm2, best_arm_index2, number_of_pulls2, expected_values2, thing_to_plot2, clairvoyants2, clair_ex2 = test_sw(
        ts_learner2, arms, adversary_bids, not only_first, active_by_influence_reward, 10000, lambdas, social_network, sw_size)

    print("Valori non solo prima posizione:")
    print(best_arm)
    print(number_of_pulls)
    print(expected_values)

    print("Valori solo prima posizione:")
    print(best_arm2)
    print(number_of_pulls2)
    print(expected_values2)


In [ ]:
print("THE BEST ARM FOR EXPERIMENT TOTAL IS:",
      arms[best_arm_index], "INDEX", best_arm_index)
print("THE BEST ARM FOR EXPERIMENT ONLY FIRST IS:",
      arms[best_arm_index2], "INDEX", best_arm_index2)

array = [i+1 for i in range(0, 20)]


plt.figure(1)
plt.errorbar(array, expected_values, yerr=(1/ts_learner.tau0)*5, fmt='o')
plt.figure(0)
plt.errorbar(array, expected_values2, yerr=(1/ts_learner2.tau0)*5, fmt='o')

plt.show()


In [ ]:
print("Here are reward and regret for the case without ad qualities, with only ads in first position")
best_values = []
j = 0
k = 0
for i in range(10000):
    k += 1
    best_values.append(clair_ex[j])
    if k == sw_size:
        k = 0
        j += 1

regret_no_qualities = [
    best_values[x] - thing_to_plot[x] for x in range(len(thing_to_plot))]
plt.figure(0)
plt.plot(thing_to_plot, 'y')
plt.plot(best_values, 'r')
print(best_arm_index)
plt.figure(1)


plt.plot(regret_no_qualities, 'g')
